🌎 Cíl projektu

Tento Python projekt simuluje počítačovou síť jako orientovaný graf, kde každý uzel reprezentuje zařízení (např. router, PC, switch), a hrany propojení mezi nimi (např. ethernetový kabel, bezdrátový přístup apod.). Projekt umí provádět algoritmickou analýzu (BFS, DFS, Dijkstra), a výsledné grafy vizualizuje jako .PNG soubory pomocí knihovny Graphviz.

Struktura projektu

main.py - Vstupní skript, zpracovává argumenty a řídí program

network_config.py - Definuje uzly (zařízení), jejich atributy a propojení

analysis.py - Obsahuje implementace algoritmů BFS, DFS, Dijkstra a zvýrazňuje hrany

diktyonphi.py - Knihovna pro práci s grafem – třídy Graph, Node, Edge a export do obrázku

Dockerfile - Umožňuje spouštění projektu v kontejneru s předinstalovaným Graphviz

requirements.txt - Seznam závislostí Pythonu pro instalaci v Dockeru

Popis main.py

Importují se moduly pro tvorbu sítě a analýzu

Funkce main() rozhoduje podle argumentu v příkazové řádce, který algoritmus se má spustit

build_network() sestaví síť

g.export_to_png() vygeneruje výstupní obrázek sítě

In [ ]:
import sys  # Modul pro práci s argumenty z příkazové řádky
from network_config import build_network  # Funkce pro sestavení grafu sítě
from analysis import bfs, dfs, dijkstra_path  # Import algoritmů z analysis.py

def main():
    print("
=== GENERUJI GRAF SÍTĚ ===")
    g = build_network()  # 🛠 Vytvoření grafu sítě na základě konfigurace

    if "--bfs" in sys.argv:
        start = sys.argv[2]  # druhý argument určuje počáteční uzel
        print(f"Spouštím BFS z uzlu {start}:")
        bfs(g, start)  # ✅ Spuštění průchodu grafem do šířky

    elif "--dfs" in sys.argv:
        start = sys.argv[2]
        print(f"Spouštím DFS z uzlu {start}:")
        dfs(g, start)  # ✅ Spuštění průchodu grafem do hloubky

    elif "--dijkstra" in sys.argv:
        start = sys.argv[2]
        end = sys.argv[3]
        print(f"Hledání nejkratší cesty z {start} do {end} pomocí Dijkstrova algoritmu:")
        dijkstra_path(g, start, end)  # ✅ Výpočet nejkratší cesty

    print("=== EXPORTUJÍ VÝSTUP ===")
    g.export_to_png("network.png")  # 🖼 Export grafu do PNG souboru

if __name__ == "__main__":
    main()  # 🔁 Zajišťuje, že se funkce spustí při přímém spuštění souboru

Co to znamená sys.argv[n]

In [ ]:
sys.argv[0] == "main.py"       # název souboru
sys.argv[1] == "--dijkstra"    # typ algoritmu
sys.argv[2] == "PC1"           # počáteční uzel
sys.argv[3] == "PC4"           # cílový uzel

Podrobný rozbor souboru network_config.py

Graph(GraphType.DIRECTED) nastaví síť jako směrovanou (orientovanou)

DEVICE_COLORS definuje barvu podle typu zařízení

Každé zařízení má svůj uzel s atributem label, type a color

add_edge() propojí uzly a přiřadí hraně váhu (weight) – např. latenci

Funkce vrací kompletně postavený graf vhodný pro analýzu nebo vizualizaci

In [ ]:
# Import třídy Graph a typu GraphType z vlastní knihovny diktyonphi
from diktyonphi import Graph, GraphType

# Slovník pro přiřazení barev jednotlivým typům zařízení
DEVICE_COLORS = {
    "router": "red",       # Červená pro routery
    "switch": "orange",    # Oranžová pro přepínače
    "client": "skyblue",   # Modrá pro klienty (PC)
    "server": "green",     # Zelená pro servery
    "ap": "violet",        # Fialová pro Access Point
    "nas": "brown",        # Hnědá pro NAS
    "mobile": "pink"       # Růžová pro mobilní zařízení
}

def build_network() -> Graph:
    '''🧠 Funkce build_network vytvoří orientovaný graf reprezentující počítačovou síť.

    Vrací:
        Graph – objekt grafu s uzly a hranami, připravený pro analýzu nebo vizualizaci
    '''
    g = Graph(GraphType.DIRECTED)  # Vytvoření orientovaného grafu (směrovaný)

    # Definice všech zařízení (uzlů) ve formě slovníku
    devices = {
        # === ROUTERY ===
        "R1": {"label": "Router 1", "type": "router"},
        "R2": {"label": "Router 2", "type": "router"},
        "R3": {"label": "Router 3", "type": "router"},

        # === PŘEPÍNAČE ===
        "SW1": {"label": "Switch 1", "type": "switch"},
        "SW2": {"label": "Switch 2", "type": "switch"},

        # === BEZDRÁTOVÝ PŘÍSTUPOVÝ BOD ===
        "AP1": {"label": "Access Point", "type": "ap"},

        # === KLIENTSKÁ ZAŘÍZENÍ (PC) ===
        "PC1": {"label": "PC 1", "type": "client"},
        "PC2": {"label": "PC 2", "type": "client"},
        "PC3": {"label": "PC 3", "type": "client"},
        "PC4": {"label": "PC 4", "type": "client"},
        "PC5": {"label": "PC 5", "type": "client"},

        # === MOBILNÍ ZAŘÍZENÍ ===
        "M1": {"label": "Smartphone", "type": "mobile"},
        "M2": {"label": "Tablet", "type": "mobile"},

        # === SERVERY ===
        "S1": {"label": "Web Server", "type": "server"},
        "S2": {"label": "Database Server", "type": "server"},

        # === SÍŤOVÉ ÚLOŽIŠTĚ (NAS) ===
        "NAS1": {"label": "NAS Storage", "type": "nas"},
    }

    # Přidání každého zařízení jako uzlu do grafu
    for node_id, attrs in devices.items():
        # Přiřazení barvy podle typu zařízení (např. "router" → "red")
        attrs["color"] = DEVICE_COLORS.get(attrs["type"], "gray")
        # Přidání uzlu do grafu s atributy (label, type, color)
        g.add_node(node_id, attrs)

    '''
    Výsledkem této smyčky je, že všechny uzly jsou přidány do grafu
    s typem, popiskem a barvou podle zařízení.
    '''

    # === DEFINICE HRAN ===
    # Každá hrana je reprezentována trojicí (zdroj, cíl, váha)
    # Váha představuje např. latenci (ms) nebo délku spoje
    edges = [
        # === PÁTEŘNÍ SPOJE MEZI ROUTERY ===
        ("R1", "R2", 1.5),
        ("R2", "R3", 1.5),
        ("R3", "R1", 1.5),

        # === PROPOJENÍ ROUTERŮ NA INFRASTRUKTURU ===
        ("R1", "SW1", 1.0),
        ("R2", "SW2", 1.0),
        ("R3", "S1", 2.0),
        ("R3", "S2", 2.0),
        ("R2", "NAS1", 2.5),

        # === PROPOJENÍ PŘEPÍNAČŮ S KLIENTY ===
        ("SW1", "PC1", 0.8),
        ("SW1", "PC2", 0.8),
        ("SW2", "PC3", 0.8),
        ("SW2", "PC4", 0.8),
        ("SW2", "PC5", 0.8),

        # === PROPOJENÍ AP A MOBILNÍCH ZAŘÍZENÍ ===
        ("SW2", "AP1", 0.5),
        ("AP1", "M1", 0.4),
        ("AP1", "M2", 0.4),
    ]

    # Přidání každé hrany do grafu s atributem "weight"
    for src, dst, weight in edges:
        g.add_edge(src, dst, {"weight": weight})

    '''
    Každá hrana je přidána do grafu s atributem:
    - weight (váha), který reprezentuje např. vzdálenost, latenci nebo rychlost přenosu
    - Orientace hran respektuje směrování (zleva doprava)
    '''

    return g  # Vrací hotový graf s uzly a hranami


Popis analysi.py

bfs() používá frontu a vyznačuje průchod grafem zelenou barvou

dfs() pracuje rekurzivně přes dfs_util() a vyznačuje fialově

dijkstra_path() hledá nejkratší cestu dle vah, zvýrazňuje modře
Zde jsou implementovány tři algoritmy:

BFS – prochází síť do šířky, barví hrany zeleně

DFS – prochází síť do hloubky, barví hrany fialově

Dijkstra – najde nejkratší cestu mezi dvěma uzly, vyznačí ji modře

Každý algoritmus využívá metodu graph.node(...).to(...) pro získání konkrétní hrany a přiřazuje atributy:

"color" – barva hrany

"penwidth" – tloušťka čáry v grafu

In [ ]:
import heapq  # knihovna pro prioritní frontu (využívá se v Dijkstrovi algoritmu)

def bfs(graph, start):
    '''
    Algoritmus Breadth-First Search (BFS) – průchod do šířky.
    Vstup: graf a počáteční uzel.
    Výstup: obarvený graf – všechny hrany, které byly navštíveny, jsou zelené.
    '''
    visited = set()             # množina navštívených uzlů
    queue = [start]             # fronta pro BFS
    visited.add(start)         # přidání počátečního uzlu

    while queue:
        current = queue.pop(0)  # odebrání prvního z fronty
        for neighbor in graph.node(current).neighbor_ids:
            if neighbor not in visited:
                visited.add(neighbor)          # označení jako navštívené
                queue.append(neighbor)         # přidání do fronty
                edge = graph.node(current).to(neighbor)
                edge["color"] = "green"         # zvýraznění hran zeleně
                edge["penwidth"] = 2           # silnější čára

def dfs_util(graph, current, visited):
    '''
    Pomocná rekurzivní funkce pro DFS.
    Navštěvuje sousedy a vybarvuje hrany fialově.
    '''
    for neighbor in graph.node(current).neighbor_ids:
        if neighbor not in visited:
            visited.add(neighbor)
            edge = graph.node(current).to(neighbor)
            edge["color"] = "purple"      # zvýraznění hran fialově
            edge["penwidth"] = 2
            dfs_util(graph, neighbor, visited)  # rekurzivní volání

def dfs(graph, start):
    '''
    Algoritmus Depth-First Search (DFS) – průchod do hloubky.
    Vstup: graf a počáteční uzel.
    Výstup: obarvený graf – cesty průchodu jsou fialové.
    '''
    visited = set()
    visited.add(start)
    dfs_util(graph, start, visited)

def dijkstra_path(graph, start, end):
    '''
    Dijkstrův algoritmus – hledání nejkratší cesty z bodu A do B.
    Používá se vážený graf (atribut 'weight' na hranách).
    Výstup: obarvená nejkratší cesta modrou barvou.
    '''
    dist = {node.id: float("inf") for node in graph}  # počáteční vzdálenosti
    prev = {}                                          # slovník předchůdců
    dist[start] = 0
    queue = [(0, start)]                              # prioritní fronta

    while queue:
        current_dist, current = heapq.heappop(queue)  # uzel s nejmenší vzdáleností
        if current == end:
            break  # konec pokud jsme v cílovém uzlu
        for neighbor in graph.node(current).neighbor_ids:
            edge = graph.node(current).to(neighbor)
            alt = current_dist + edge["weight"]       # nová vzdálenost
            if alt < dist[neighbor]:
                dist[neighbor] = alt
                prev[neighbor] = current              # aktualizace předchůdce
                heapq.heappush(queue, (alt, neighbor))

    # Rekonstrukce nejkratší cesty
    path = []
    node = end
    while node in prev:
        path.insert(0, node)  # vkládáme zpětně
        node = prev[node]
    path.insert(0, start)

    # Vybarvení cesty modře
    for i in range(len(path)-1):
        edge = graph.node(path[i]).to(path[i+1])
        edge["color"] = "blue"
        edge["penwidth"] = 3

Popis diktionphi.py

In [ ]:
import enum  # pro definici typu grafu (orientovaný vs. neorientovaný)
import subprocess  # umožňuje volání externích příkazů (např. Graphviz)
from typing import Dict, Hashable, Any, Optional, Iterator, Tuple  # typové anotace

# Výčtový typ pro typ grafu
class GraphType(enum.Enum):
    DIRECTED = 0  # orientovaný graf (směrovaný)
    UNDIRECTED = 1  # neorientovaný graf

# Třída pro reprezentaci hrany mezi dvěma uzly
class Edge:
    def __init__(self, src: 'Node', dest: 'Node', attrs: Dict[str, Any]):
        '''
        Vytvoří hranu mezi dvěma uzly.
        src – výchozí uzel
        dest – cílový uzel
        attrs – slovník atributů, např. váha, barva
        '''
        self.src = src
        self.dest = dest
        self._attrs = attrs

    def __getitem__(self, key: str) -> Any:
        # Umožňuje přístup jako edge["weight"]
        return self._attrs[key]

    def __setitem__(self, key: str, val: Any) -> None:
        # Umožňuje nastavení hodnoty jako edge["color"] = "blue"
        self._attrs[key] = val

# Třída pro reprezentaci jednoho uzlu v grafu
class Node:
    def __init__(self, graph: 'Graph', node_id: Hashable, attrs: Dict[str, Any]):
        '''
        graph – reference na celý graf
        node_id – unikátní identifikátor (např. "PC1")
        attrs – atributy uzlu, např. label nebo barva
        '''
        self.id = node_id
        self.graph = graph
        self._attrs = attrs
        self._neighbors: Dict[Hashable, Dict[str, Any]] = {}  # sousedé a jejich hrany

    def __getitem__(self, item: str) -> Any:
        return self._attrs[item]

    def __setitem__(self, item: str, val: Any) -> None:
        self._attrs[item] = val

    def to(self, dest: Hashable | 'Node') -> Edge:
        '''
        Vrací hranu z aktuálního uzlu do cílového.
        '''
        dest_id = dest.id if isinstance(dest, Node) else dest
        if dest_id not in self._neighbors:
            raise ValueError(f"No edge from {self.id} to {dest_id}")
        return Edge(self, self.graph.node(dest_id), self._neighbors[dest_id])

    @property
    def neighbor_ids(self) -> Iterator[Hashable]:
        # Vrací ID všech sousedních uzlů
        return iter(self._neighbors)

    @property
    def out_degree(self) -> int:
        # Počet výstupních hran z uzlu
        return len(self._neighbors)

# Hlavní třída pro práci s grafem
class Graph:
    def __init__(self, type: GraphType):
        '''
        Inicializuje graf jako orientovaný nebo neorientovaný.
        '''
        self.type = type
        self._nodes: Dict[Hashable, Node] = {}

    def add_node(self, node_id: Hashable, attrs: Optional[Dict[str, Any]] = None) -> Node:
        '''
        Přidá nový uzel do grafu.
        '''
        if node_id in self._nodes:
            raise ValueError(f"Node {node_id} already exists")
        node = Node(self, node_id, attrs or {})
        self._nodes[node_id] = node
        return node

    def add_edge(self, src_id: Hashable, dst_id: Hashable, attrs: Optional[Dict[str, Any]] = None):
        '''
        Přidá hranu mezi dvěma uzly, včetně atributů jako váha nebo barva.
        '''
        attrs = attrs or {}
        if src_id not in self._nodes:
            self.add_node(src_id)
        if dst_id not in self._nodes:
            self.add_node(dst_id)
        self._set_edge(src_id, dst_id, attrs)
        if self.type == GraphType.UNDIRECTED:
            self._set_edge(dst_id, src_id, attrs)

    def _set_edge(self, src_id: Hashable, dst_id: Hashable, attrs: Dict[str, Any]):
        # Vnitřní metoda pro přidání směrové hrany
        if dst_id in self._nodes[src_id]._neighbors:
            raise ValueError(f"Edge {src_id}→{dst_id} already exists")
        self._nodes[src_id]._neighbors[dst_id] = attrs

    def node(self, node_id: Hashable) -> Node:
        return self._nodes[node_id]

    def node_ids(self) -> Iterator[Hashable]:
        return iter(self._nodes.keys())

    def __iter__(self) -> Iterator[Node]:
        return iter(self._nodes.values())

    def to_dot(self, label_attr="label", weight_attr="weight") -> str:
        '''
        Vygeneruje DOT zápis grafu pro Graphviz.
        Obsahuje labely, barvy, tloušťku hran atd.
        '''
        lines = []
        connector = "->" if self.type == GraphType.DIRECTED else "--"
        lines.append("digraph G {" if self.type == GraphType.DIRECTED else "graph G {")

        for node_id in self.node_ids():
            node = self.node(node_id)
            label = node[label_attr] if label_attr in node._attrs else str(node_id)
            color = node._attrs.get("color", "black")
            lines.append(f'    "{node_id}" [label="{label}", fillcolor="{color}", style=filled];')

        seen = set()
        for node in self:
            for dst_id in node.neighbor_ids:
                if self.type == GraphType.UNDIRECTED and (dst_id, node.id) in seen:
                    continue
                seen.add((node.id, dst_id))
                edge = node.to(dst_id)
                label = edge._attrs.get(weight_attr, "")
                color = edge._attrs.get("color", "black")
                penwidth = edge._attrs.get("penwidth", 1)
                lines.append(f'    "{node.id}" {connector} "{dst_id}" [label="{label}", color="{color}", penwidth={penwidth}];')

        lines.append("}")
        return "\n".join(lines)

    def export_to_png(self, filename: str):
        '''
        Spustí Graphviz a uloží graf jako PNG.
        '''
        dot = self.to_dot()
        subprocess.run(["dot", "-Tpng", "-o", filename], input=dot, text=True, check=True)
